### Imports

In [20]:
%pip install openai --upgrade
%pip install pandas

In [ ]:
import os
import csv
import openai
import pandas as pd

### OPENAI API Keys

In [1]:
openai.api_key = 'PLEASE_GET_YOUR_KEY_AND_MAKE_SURE_YOU_HAVE_SAVINGS_IN_OPENAI'

### Loading Dataset

In [43]:
validation_set = "validation.tsv"
df = pd.read_csv(validation_set, sep='\t', quoting=csv.QUOTE_NONE)
sorted_df = df.sort_values(by='tweet_id')

In [44]:
text = sorted_df.text.values
ids = sorted_df.tweet_id.values
truelabel = sorted_df.label.values

In [45]:
validation_set = "the_predict_result.tsv"
testdf = pd.read_csv(validation_set, sep='\t', quoting=csv.QUOTE_NONE)

In [46]:
testid = testdf.tweet_id.values
label = testdf.label.values

In [ ]:
# Compared with the original performance
from sklearn.metrics import classification_report
print(classification_report(truelabel, label, target_names=['negative', 'positive']))

#### Constraint

In [49]:
testtext = []
testid1 = []
for i, v in enumerate(label):
    if v == 1:
        testtext.append(text[i])
        testid1.append(testid[i])

In [ ]:
def find_longest_string_length(lst):
    longest_str = max(lst, key=len)
    return len(longest_str)

result = find_longest_string_length(text)
print("The longest text length is:", result)

In [ ]:
def count_words_in_longest_string(lst):
    longest_str = max(lst, key=len)
    words_in_longest_str = len(longest_str.split())
    return words_in_longest_str

result = count_words_in_longest_string(text)
print("The largest number of words is:", result)

In [ ]:
def average_word_count(lst):
    total_words = 0
    total_strings = len(lst)
    for string in lst:
        words_in_string = len(string.split())
        total_words += words_in_string
    average_words = total_words / total_strings
    return average_words

result = average_word_count(text)
print("Average word number is:", result)

### Prompt

In [ ]:
prompt = '''
You are given a tweet which is classified to be tweet that self-report a COVID-19 diagnosis.
Your job is to find out whether or not the input tweet is truly self-report a COVID-19 diagnosis according to the following rules: 
1. A postitive test, clinical diagnosis, or hospitalization will all be counted as True.
2. A tweet is not self-reporting a COVID-19 diagnosis or irrelavent, return False.
If the label is 0, please return True.

Here are some examples: 
Input: I've been diagnosed with Covid, Label: 0
Output: False, because the user explicitly said he has Covid. But the given label is 0.
Input My therapists diagnosed me with Covid-19. Label 1
Output: True, because the user explicitly said he has Covid.

Your output should only be True or False with reasoning sentences that can show the proof step by step within 30 words. 
You should return True for most uncertain cases and be careful to return False.
'''
print(prompt)

In [58]:
prompts = testtext
id1 = testid1    

In [60]:
def prepare_message(uinput):
    #preparing the prompt for OpenAI 
    role="user"
    #prompt="Where is Tahiti located?" #maintenance or if you do not want to use a microphone
    line = {"role": role, "content": uinput}
    
    #creating the mesage
    assert1={"role": "system", "content": "You are a highly intelligent and accurate COVID-19 diagnosis binary classification result explain system."}
    assert2={"role": "assistant", "content": prompt}
    assert3=line
    iprompt = []
    iprompt.append(assert1)
    iprompt.append(assert2)
    iprompt.append(assert3)

    return iprompt

### GPT4 inference if you have

In [17]:
# # we can use gpt3.5
# uinput="Is the following tweet postive or negative:She loved the new Batman movie! But I agreed with that critic we just heard."
# iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
# response=openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=iprompt) #ChatGPT dialog
# text=response["choices"][0]["message"]["content"] #response in JSON
# print("ChatGPT response:",text)

In [18]:
# # I have applied for gpt4 but have not got the permission
# uinput=inputs
# iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
# response=openai.ChatCompletion.create(model="gpt-4",messages=iprompt) #ChatGPT dialog
# text=response["choices"][0]["message"]["content"] #response in JSON
# print("ChatGPT response:",text)

### GPT3.5 inference pipeline

In [61]:
def gpt(uinput):
    iprompt=prepare_message(uinput) #preparing the messages for ChatGPT
    response=openai.ChatCompletion.create(model="gpt-3.5-turbo",messages=iprompt) #ChatGPT dialog
    text=response["choices"][0]["message"]["content"] #response in JSON
    print("ChatGPT response:",text)
    return text

In [64]:
import time
predicts = []
for t, i in enumerate(prompts):
    if (t + 1) % 3 == 0:
        time.sleep(60)  # 停顿一分钟
    
    print(t+ 1,':', testid1[t])
    inputs = i
    predict = str(gpt(inputs))
    predicts.append(predict)

1 : 1325856099606663168
ChatGPT response: True. The tweet self-reports an asymptomatic COVID-19 diagnosis and expresses concerns about potentially spreading the virus to patients in a hospital setting.
2 : 1325865731142144002
ChatGPT response: False. The input tweet does not self-report a COVID-19 diagnosis.
3 : 1328145791802888196
ChatGPT response: True. The sentence "Tested positive for COVID" indicates a self-report of COVID-19 diagnosis, confirming the need for a vaccine.
4 : 1336193603178274817
ChatGPT response: True. The tweet self-reports a COVID-19 diagnosis with mention of testing positive for COVID-19 and experiencing symptoms such as zero stamina and a cough.
5 : 1338628172020150276
ChatGPT response: True. The tweet indicates that the person had COVID-19 in July, gets tested for antibodies regularly, and recently tested positive for antibodies. This suggests a COVID-19 diagnosis.
6 : 1346614922579423232
ChatGPT response: True. The user mentions testing positive for COVID-19 

### Correction

In [ ]:
correction = []
for i , v in enumerate(predicts):
    if "false" in v or "False" in v or "FALSE" in v:
        correction.append(id1[i])

In [ ]:
for i , v in enumerate(ids):
    if v in correction:
        print('correct id', v)
        label[i] = 0

In [ ]:
# Compare the performance now
print(classification_report(truelabel, label, target_names=['negative', 'positive']))

In [67]:
# Save the results
def save_lists_to_csv(list1, list2, filename):
    data_dict = {
        'id': list1,
        'bool': [item.split()[0] for item in list2],
        'explanation': [' '.join(item.split()[1:]) for item in list2]
    }

    df = pd.DataFrame(data_dict)
    df.to_csv(filename, index=False)


csv_filename = 'ALEX_task1_label1_corrected.csv'
save_lists_to_csv(testid1, predicts, csv_filename)
